Create a test sql database from titanic dataset.

https://python.langchain.com/docs/use_cases/sql/csv/

In [1]:
import pandas as pd
from pyprojroot import here

In [2]:
df = pd.read_csv(here("/Users/main/Desktop/chatbot 2/data/todos_restaurantes.csv"),index_col=0)
# print(df.shape)

# # print(df.columns.tolist())
# display(df.head(3))

### **SQL**

Using SQL to interact with CSV data is the recommended approach because it is easier to limit permissions and sanitize queries than with arbitrary Python.

Most SQL databases make it easy to load a CSV file in as a table (DuckDB, SQLite, etc.). Once you’ve done this you can use all of the chain and agent-creating techniques outlined in the SQL use case guide. Here’s a quick example of how we might do this with SQLite:

In [3]:
from sqlalchemy import create_engine, inspect
db_path="all_data.db"
engine = create_engine(f"sqlite:///{db_path}")

connection = engine.connect()

inspector = inspect(engine)

table_names = inspector.get_table_names()

In [4]:
# for table_name in table_names:
#     print(f"Information for table: {table_name}")
    
#     print(f"Schema: {inspector.get_schema_names()}")
    
#     columns = inspector.get_columns(table_name)
#     for column in columns:
#         print(f"Column: {column['name']} Type: {column['type']}")
   
#     pk_constraint = inspector.get_pk_constraint(table_name)
#     print(f"Primary Key Constraint: {pk_constraint}")

#     foreign_keys = inspector.get_foreign_keys(table_name)
#     print(f"Foreign Keys: {foreign_keys}")

# connection.close()

In [5]:
from langchain_community.utilities import SQLDatabase

db= SQLDatabase(engine=engine)

In [6]:
print(db.dialect)
print(db.get_usable_table_names())

sqlite
['data_restaurants']


For multiple csv files, we can create a sql with multiple tables:
```
df1.to_sql("csv1_name", engine, index=False)
df2.to_sql("csv2_name", engine, index=False)
```

In [9]:

db.run("SELECT * FROM data_restaurants WHERE rating < 2 Limit 2;")

'[(None, 0, 1, \'600 00 00 00\', \'Cafetería Amazonas\', 1, "[{\'close\': {\'day\': 0, \'time\': \'2100\'}, \'open\': {\'day\': 0, \'time\': \'1100\'}}, {\'close\': {\'day\': 1, \'time\': \'2300\'}, \'open\': {\'day\': 1, \'time\': \'0800\'}}, {\'close\': {\'day\': 2, \'time\': \'2300\'}, \'open\': {\'day\': 2, \'time\': \'0800\'}}, {\'close\': {\'day\': 3, \'time\': \'2300\'}, \'open\': {\'day\': 3, \'time\':...", "[\'Monday: 8:00\\\\u202fAM\\\\u2009–\\\\u200911:00\\\\u202fPM\', \'Tuesday: 8:00\\\\u202fAM\\\\u2009–\\\\u200911:00\\\\u202fPM\', \'Wednesday: 8:00\\\\u202fAM\\\\u2009–\\\\u200911:00\\\\u202fPM\', \'Thursday: 8:00\\\\u202fAM\\\\u2009–\\\\u200911:00\\\\u202fPM\', \'Friday: 8:00\\\\u202fAM\\\\u2009–\\\\u200911:00\\\\u202fPM\', \'Saturday:...", \'ChIJKdFsc7soQg0RlIQ-k7uz4BU\', None, 1.0, "[{\'author_name\': \'Cebollo\', \'author_url\': \'https://www.google.com/maps/contrib/112758889190499253904/reviews\', \'language\': \'en-US\', \'original_language\': \'es\', \'profile_photo_

### **Create an agent to interact with the Database**

In [10]:
import os
from dotenv import load_dotenv
import warnings
from langchain_openai import ChatOpenAI

warnings.filterwarnings("ignore")
OPEN_AI_API_KEY = os.getenv("OPEN_AI_API_KEY")

In [12]:

# model_name = os.getenv("gpt_deployment_name")
# azure_openai_api_key = os.environ["OPENAI_API_KEY"]
# azure_openai_endpoint = os.environ["OPENAI_API_BASE"]

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=1,
    model_name="amiguis")

In [13]:
print(llm)

client=<openai.resources.chat.completions.Completions object at 0x10cc6ef90> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10cc76ba0> root_client=<openai.OpenAI object at 0x10c96ef90> root_async_client=<openai.AsyncOpenAI object at 0x10cc6f0e0> temperature=1.0 model_kwargs={} openai_api_key=SecretStr('**********')


In [14]:
from langchain_community.agent_toolkits import create_sql_agent
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [17]:
agent_executor.invoke({"input": "What is the average rating in the data_restaurants table?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


data_restaurants
Invoking: `sql_db_schema` with `{'table_names': 'data_restaurants'}`



CREATE TABLE data_restaurants (
	curbside_pickup INTEGER, 
	delivery INTEGER, 
	dine_in INTEGER, 
	formatted_phone_number TEXT, 
	name TEXT, 
	open_now INTEGER, 
	periods TEXT, 
	weekday_text TEXT, 
	place_id TEXT, 
	price_level REAL, 
	rating REAL, 
	reviews TEXT, 
	serves_beer INTEGER, 
	serves_breakfast INTEGER, 
	serves_brunch INTEGER, 
	serves_dinner INTEGER, 
	serves_lunch INTEGER, 
	serves_vegetarian_food INTEGER, 
	serves_wine INTEGER, 
	takeout INTEGER, 
	url TEXT, 
	user_ratings_total REAL, 
	website TEXT, 
	wheelchair_accessible_entrance INTEGER, 
	reservable INTEGER, 
	overview TEXT, 
	error_message TEXT, 
	types TEXT, 
	vicinity TEXT, 
	"geometry.location.lat" REAL, 
	"geometry.location.lng" REAL
)

/*
3 rows from data_restaurants table:
curbside_pickup	delivery	dine_in	formatted_phone_number	name	

{'input': 'What is the average rating in the data_restaurants table?',
 'output': 'The average rating in the data_restaurants table is approximately 4.18.'}